In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
x.q[joint1]

7-element Array{Float64,1}:
  0.515233 
 -0.476383 
  0.711903 
 -0.0280851
  0.852427 
  0.11959  
  0.290369 

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 2.654164118767531 rad about [0.01711590111450171,0.48462351865789466,0.8745553676541468], translation: Vec(0.7786311610638958,0.595382446928555,0.5128880153022659)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(0.7770554286905658,0.508064330817523,1.3091735156023834)

In [12]:
mass(mechanism)

0.9864701841356995

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(0.5815418117397658,0.2090143498442637,1.6070218399302552)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(0.18604788833401087,-0.560022703452804,0.7282868499088617), linear: Vec(1.0521911177392687,-0.12454623433658552,-0.3645632960013767)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
geometric_jacobian(x, path(mechanism, body2, world))

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
 -0.25067   -0.176184   0.23889   …  -0.0       -0.0       -0.0       
  0.754542   0.811152   0.584826     -0.0       -0.0       -0.0       
 -0.981251  -0.557667   0.775184     -0.0       -0.0       -0.0       
 -1.41766   -1.29268    0.286223     -0.176184   0.23889   -0.95493   
  0.167806   0.361021  -0.507143      0.811152   0.584826  -0.00335412
  0.491191   0.933519   0.294401  …  -0.557667   0.775184   0.296813  

In [17]:
mass_matrix(x)

7x7 Array{Float64,2}:
  0.830204     -1.12623     -0.936676     …   0.012505      1.07023    
 -1.12623       1.87081      2.36519         -0.630463     -0.599313   
 -0.936676      2.36519      3.3346           5.55112e-17  -0.968177   
 -8.32667e-17  -0.267153    -0.012505        -3.33067e-16   0.000765185
  0.267153     -1.2837e-16   0.630463         1.94289e-16   0.00682552 
  0.012505     -0.630463     2.77556e-17  …   0.98647      -0.0306541  
  1.07023      -0.599313    -0.968177        -0.0306541     1.57554    

In [18]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
 -0.559489    1.46341     2.25764   …   0.366765  -0.153091    -0.584272  
  0.131544   -0.780604   -0.288003      0.285694   1.46077     -0.551941  
  1.52306    -1.90987    -3.5195       -0.328564  -0.47603      1.37625   
 -0.0518788  -0.649116   -0.152814     -0.235658   0.94201     -0.0307683 
 -0.156196    0.214587   -0.358568     -0.576913   0.00330874  -0.00471524
 -0.210805    0.0381471  -0.495698  …  -0.764696  -0.292797     0.00423423

In [19]:
A * velocity_vector(x)

Momentum expressed in "world":
angular: Vec(1.90250345676739,-0.29373766319311817,-0.9756742846430437), linear: Vec(-0.18284274724980085,-1.1100254449680353,-0.5283810178664314)

In [20]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(0.39740489126249534,-0.5654523251610813,0.4987156727102909), linear: Vec(1.2082098956259908,0.45726669691470173,0.6095183011820076)

In [21]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(-0.7584358529287257,-0.2746732440307963,-0.017462712412940146), linear: Vec(-0.9125962029363397,-1.0159130469008213,0.49197086439166454)

In [22]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(x, v̇)

7-element Array{Float64,1}:
  3.08046 
 -0.462364
  5.76816 
 -5.21536 
  8.5445  
  3.44151 
  0.638276

In [23]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [24]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body3, Edge: joint3
      Vertex: body4, Edge: joint4
        Vertex: body8, Edge: joint8
          Vertex: body10, Edge: joint10
      Vertex: body6, Edge: joint6
  Vertex: body2, Edge: joint2
    Vertex: body5, Edge: joint5
      Vertex: body9, Edge: joint9
      Vertex: body11, Edge: joint11
    Vertex: body7, Edge: joint7